In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import LSTM, Input, Dense, LSTMCell, RNN
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
def read_data(file_location):
    df = pd.read_csv(file_location)
    return df

In [ ]:
def preprocess_data(df, aggregation='H', ws=24, number_of_predicted_days=2):

    df.dropna(inplace=True)

    df.drop_duplicates(subset=['source_ts'], inplace=True)

    datetime_series = pd.to_datetime(df['source_ts'])
    datetime_index = pd.DatetimeIndex(datetime_series.values)
    df=df.set_index(datetime_index)
    df.drop('source_ts',axis=1,inplace=True)

    df=df.asfreq(freq='S', method='ffill')

    lastDay = df.index[-1].strftime('%Y-%m-%d')
    df = df.loc[:lastDay].iloc[:-1 , :]
    df

    prediction_in_future_time = ws * number_of_predicted_days
    
    df_resampled = df.resample(aggregation).sum()
    df_resampled
    
    df = df_resampled

    index_of_start_prediction = 0 - prediction_in_future_time
    day_of_start_prediction = df.index[index_of_start_prediction].strftime('%Y-%m-%d')
    index_of_end_train = index_of_start_prediction - ws
    day_of_end_train = df.index[index_of_end_train].strftime('%Y-%m-%d')

    TRAIN_END = day_of_end_train
    TEST_START = day_of_start_prediction

    training_set_df = df.loc[:TRAIN_END]

    test_set_df = df.loc[TEST_START:]

    return training_set_df, test_set_df

In [ ]:
def create_model(no_input_features=1, no_output_features=1, layers=[6]):
    
    n_layers = len(layers)
    
    ## Encoder
    encoder_inputs = Input(shape=(None, no_input_features))
    lstm_cells = [LSTMCell(hidden_dim) for hidden_dim in layers]
    encoder = RNN(lstm_cells, return_state=True)
    encoder_outputs_and_states = encoder(encoder_inputs)
    encoder_states = encoder_outputs_and_states[1:]
    
    ## Decoder
    decoder_inputs = Input(shape=(None, no_output_features))
    decoder_cells = [LSTMCell(hidden_dim) for hidden_dim in layers]
    initial_states = encoder_states
        
    decoder_lstm = RNN(decoder_cells, return_sequences=True, return_state=True)

    decoder_outputs_and_states = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_outputs = decoder_outputs_and_states[0]

    decoder_dense = Dense(no_output_features, activation='relu') 
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model([encoder_inputs,decoder_inputs], decoder_outputs)
    return model
    

In [ ]:
def create_input_sequences(data, ws):
    x_train, y_train = [], []

    for i in range(ws, len(data)-ws-1):
        x_train.append(data[i-ws:i, 0:1])
        y_train.append(data[i+1:i+1+ws, 0:1])
    
    
    return np.array(x_train), np.array(y_train)

In [ ]:
def train_model(model, train_dataset, epochs=10, batch_size=32):
    
    model.compile(Adam(), loss='mean_squared_error')
    
    input_sequence, output_sequence = create_input_sequences(train_dataset, 24)
    
    encoder_input = input_sequence
    decoder_target = output_sequence
    decoder_input = np.zeros(decoder_target.shape)
    
    history = model.fit([encoder_input, decoder_input], decoder_target,
                       batch_size=batch_size,
                       epochs=epochs,
                       validation_split=0.1,
                       shuffle=False)
        
    return model

In [ ]:
def make_predictions(model, initial_batch, input_len=24, output_len=24, ws=24, prediction_in_future_time=48):
    prediction_test = []
    batch = []
    batch = np.array([initial_batch[-ws:, 0:1]])  
    print(batch.shape)
    
    for _ in range(2):
        input_seq_test = batch
        element = np.array([[[0.]]]) 
        decoder_input_test = np.stack([element]*24, axis=1)
        prediction = model.predict([input_seq_test, decoder_input_test])[0]
        
        print(prediction)

        batch = np.array([np.append(batch[0,24:], prediction, axis=0)])
        
        prediction_test.append(prediction)
        
    return np.array(prediction_test)

In [ ]:
def plot_results(test_set, predictions):
    plt.rcParams["figure.figsize"] = (40,3)
    plt.plot(test_set, color='green', label='Actual value')
    plt.plot(predictions, color='orange', label='Predicted value')
    plt.legend()
    plt.show()

In [ ]:
def main(file_location):
    
    df = read_data(file_location)
    
    training_set_df, test_set_df = preprocess_data(df)
    
    training_set = training_set_df.values
    test_set = test_set_df.values
    
    sc = MinMaxScaler(feature_range=(0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    
    model = create_model(layers=[64])
    model = train_model(model, train_dataset=training_set_scaled, epochs=20)
    
    batch_init = training_set_scaled[-ws:]
    predictions = make_predictions(model, batch_init)
        
    merged_predictions = [item for sublist in predictions for item in sublist]
    predictions = [[item.item()] for item in merged_predictions]
    
    predictions = sc.inverse_transform(predictions)
    
    rmse, rsquare = evaluate_model(test_set, predictions)
    print("RMSE:", rmse)
    print("R-squared:", rsquare)
    
    plot_results(test_set,predictions)

In [ ]:
def evaluate_model(test_set, predictions):
    rmse = np.sqrt(mean_squared_error(test_set, predictions))
    rsquare = r2_score(test_set, predictions)
    return rmse, rsquare

In [ ]:
ws = 24
prediction_in_future_time = ws * 2
main("./testData/Dataset1.csv")